In [61]:
import requests
import json
import pandas as pd
import xmltodict
import random
from pprint import pprint
from api_keys import api_key
addresses = pd.read_csv("../data/rawdata/All_Addresses.csv")
a = pd.read_csv("../data/rawdata/Harris_County.csv",header=None)
zips =  list(a[0])

In [53]:
base = "https://www.zillow.com/webservice/GetDeepSearchResults.htm?"
zwsid = api_key
address = "Both address and Zip available on the random addresses csv"
citystatezip = "zipcode ONLY"
query = f"{base}zws-id={zwsid}&address={address}&citystatezip={citystatezip}"

In [156]:
print(len(zips))
tries = 0

146


In [220]:
#group_Adds = pd.DataFrame(columns=["zip", "address", "lat", "lng","house area","lot area","high","low","estimate","month value change","property tax"])
group_Totals = pd.DataFrame(columns=["zip", "address", "lat", "lng","house area","lot area","high","low","estimate","month value change","property tax"])
row = [0,1,2,3,4,5,6,7,8,9,10]
gb = {"good":[],"bad":{}}
manual = 28
ID = "X1-ZWz1gz687fgzyj_9r65p"


#manually do groups of five to catch when zillow goes bad
for zippy in zips[(5*(manual-1)):]:#(5*(manual-1)):(5*manual),  zips,using zips[i] and len(total_Addresses) doesn't work cause there was another random error that caused a retry
    #in the middle of assignment of values to the dataframe. While not affecting the actual list of values, total_Addresses
    #now has a single repeat value for one of the functions. Total addresses will now be done with the count function
    #of individual groupby after county_Zip_Adds has been made, though it is less efficient
    
    #Resets the zipcode dataframe
    group_Totals = pd.DataFrame(columns=["zip", "address", "lat", "lng","house area","lot area","high","low","estimate","month value change","property tax"])
    
    #Gets All recorded addresses of a zipcode
    df = addresses.loc[addresses["zip"] == zippy]
    #Finds only exact matches as \\bWord\\b finds only the word, and not anything that contains it, unless other chars are buffered by a space away from it
    df = df.loc[(df["address"].str.contains("\\bUNIT\\b|\\bAPT\\b|\\bSTE\\b|\\bBSMT\\b|\\bFL\\b|\\bHNGR\\b|\\bLOT\\b|\\bRM\\b|\\bSLIP\\b|\\bSTE\\b|\\bTRLR\\b") == False)]
    #Finds things that start with a number, as streets or whatever can't be searched in zillow
    df = df.loc[df["address"].str.startswith(("0","1","2","3","4","5","6","7","8","9"))]
    #This reset index is almost definitely superfluous. Too scared to touch it at the moment, don't wanna mess it up now it's finally working
    df = df.reset_index(drop=True)
    length = len(df.index)
    
    #Only runs if there are at least 500 possible results
    if length >= 500:
        #Shuffles Dataframe
        rands = random.sample(range(0,length),length)
        df = df.iloc[rands]
        df = df.reset_index(drop=True)
        
        for i in range(0,length):#length
            
            #breaks when 500 records have been collected, incomplete or not. All have zestimate
            if len(group_Totals.index) >= 500:
                break
            
            address = df["address"][i].split(',')[0]
            row[1] = df["address"][i]
            citystatezip = str(zippy)
            row[0] = citystatezip
            row[2] = df["lat"][i]
            row[3] = df["lng"][i]
            query = f"{base}zws-id={ID}&address={address}&citystatezip={citystatezip}"
            #print(requests.get(query))
            info = requests.get(query)
            dictInfo = xmltodict.parse(info.text)
            jsonInfo = json.loads(json.dumps(dictInfo))
            #print(jsonInfo["SearchResults:searchresults"]["message"]["code"])
            if jsonInfo["SearchResults:searchresults"]["message"]["code"] == "0":
                cliff = jsonInfo["SearchResults:searchresults"]["response"]["results"]["result"]
                try:
                    row[4] = cliff["finishedSqFt"]
                except:
                    row[4] = float("nan")
                try:
                    row[5] = cliff["lotSizeSqFt"]
                except:
                    row[5] = float("nan")
                try:
                    row[6] = cliff["zestimate"]["valuationRange"]["high"]["#text"]
                    row[7] = cliff["zestimate"]["valuationRange"]["low"]["#text"]
                except:
                    row[6] = float("nan")
                    row[7] = float("nan")
                try:
                    row[8] = cliff["zestimate"]["amount"]["#text"]
                #except KeyError:
                    #print("Key error")
                    #pprint(jsonInfo)
                    #break
                except:
                    #print("No value listing")
                    continue
                    #print("Not Key Error")
                    #pprint(jsonInfo)
                    #break
                try:
                    row[9] = cliff["zestimate"]["valueChange"]["#text"]
                except:
                    row[9] = float("nan")
                try:
                    row[10] = cliff["taxAssessment"]
                except:
                    row[10] = float("nan")
                #print(row)
                group_Totals.loc[i] = row
    tries += i
    if len(group_Totals.index) >= 500:
        gb["good"].append(str(zippy))
        group_Totals.to_csv(f"../data/rawdata/Property_Values/{str(zippy)}.csv",index=False,encoding = "utf-8")
    else:
        gb["bad"][str(zippy)] = len(group_Totals.index)
        group_Totals.to_csv(f"../data/rawdata/Property_Values/failed_{str(zippy)}.csv",index=False,encoding = "utf-8")

#outputs which zipcodes were good and bad, and how many records are in each bad
with open(f"../data/rawdata/Property_Values/Manual_Tries/County_Zips_{manual}.json","w") as outs:
        json.dump(gb,outs)
#group_Totals        
#group_Adds = group_Adds.reset_index(drop=True)

In [221]:
print(tries)
print(gb)
#77204,217,249,251,266,268,271,284,289 are all too small and will fail. It's fine.
#77507 should work

106526
{'good': ['77520', '77530', '77532', '77536', '77546', '77547', '77562', '77571', '77586', '77587', '77598'], 'bad': {}}


In [216]:
df = addresses.loc[addresses["zip"] == 77507]
df.head()

,zip,address,lat,lng
895569,77507,"12435 RED BLUFF RD, PASADENA, TX 77507",29.591431,-95.053132
895570,77507,"9311 BAY AREA BLVD, PASADENA, TX 77507",29.602694,-95.069038
895571,77507,"9501 BAY AREA BLVD, PASADENA, TX 77507",29.604062,-95.065751
895572,77507,"9810 BAY AREA BLVD, PASADENA, TX 77507",29.606006,-95.059925
895573,77507,"9801 BAY AREA BLVD, PASADENA, TX 77507",29.606336,-95.060312


In [198]:
#print(addresses.head())
pprint(jsonInfo)

{'SearchResults:searchresults': {'@xmlns:SearchResults': 'http://www.zillow.com/static/xsd/SearchResults.xsd',
                                 '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
                                 '@xsi:schemaLocation': 'http://www.zillow.com/static/xsd/SearchResults.xsd '
                                                        'https://www.zillowstatic.com/vstatic/9fccb87/static/xsd/SearchResults.xsd',
                                 'message': {'code': '0',
                                             'text': 'Request successfully '
                                                     'processed'},
                                 'request': {'address': '12018 HUNTINGTON PARK '
                                                        'DR',
                                             'citystatezip': '77099'},
                                 'response': {'results': {'result': {'FIPScounty': '48201',
                                              

In [51]:
pprint(json.loads(json.dumps(dictInfo)))

{'SearchResults:searchresults': {'@xmlns:SearchResults': 'http://www.zillow.com/static/xsd/SearchResults.xsd',
                                 '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
                                 '@xsi:schemaLocation': 'http://www.zillow.com/static/xsd/SearchResults.xsd '
                                                        'https://www.zillowstatic.com/vstatic/9fccb87/static/xsd/SearchResults.xsd',
                                 'message': {'code': '0',
                                             'text': 'Request successfully '
                                                     'processed'},
                                 'request': {'address': '4838 ROCKWOOD DR',
                                             'citystatezip': '77004'},
                                 'response': {'results': {'result': {'FIPScounty': '48201',
                                                                     'address': {'city': 'Houston',
             

In [91]:
pprint(jsonInfo)

{'SearchResults:searchresults': {'@xmlns:SearchResults': 'http://www.zillow.com/static/xsd/SearchResults.xsd',
                                 '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
                                 '@xsi:schemaLocation': 'http://www.zillow.com/static/xsd/SearchResults.xsd '
                                                        'https://www.zillowstatic.com/vstatic/9fccb87/static/xsd/SearchResults.xsd',
                                 'message': {'code': '0',
                                             'text': 'Request successfully '
                                                     'processed'},
                                 'request': {'address': '201 CAROLINE ST',
                                             'citystatezip': '77002'},
                                 'response': {'results': {'result': {'FIPScounty': None,
                                                                     'address': {'city': 'Houston',
                 

In [65]:
df = pd.DataFrame({"Hey":[0,1,2,3,4,5]})
length = len(df.index)
rands = random.sample(range(0,length),length)
df.iloc[rands]
df = pd.DataFrame(columns=["Hey"])
df

,Hey


In [86]:
cliff["zestimate"]["valuationRange"]["high"]["#text"]

'392616'

In [88]:
row[6] = cliff["zestimate"]["valuationRange"]["high"]["#text"]
row[6]

'392616'